### 高斯朴素贝叶斯
$$ P(y|x) = \frac{P(x|y)P(y)}{P(x)} $$
由上面的贝叶斯公式, 我们可以根据特征x给出类别y的判别概率.  P(x|y)是似然, P(y)是先验. 放到上面的二分类问题里, P(x|y)就是对类别y的所有样本, 估计出的一个条件概率分布. 而P(y)一般是常数, 可以直接计算不同类别样本在数据集中所占的比例得到. 在决策时, 我们对每个类别用贝叶斯公式计算P(y|x)并取最大. 这就是贝叶斯决策, 只要我们能得到精确的似然概率密度函数P(x|y)就能做出让错误率最小的决策.  
如果x的特征有很多种, 维度很高, 概率密度的空间是很大很复杂的, 我们可能很难估计出一个合适的P(x|y)的形式. 这时我们可以借助朴素贝叶斯假设, 认为条件独立, 就能用$P(x|y) \simeq \prod_i P(x_i|y) $用单个维度的分布的乘积估计高维情况下的似然概率密度P(x|y).

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def gaussian(X, mu, sigma):
    num = np.exp(-(X-mu)**2/2/sigma**2)
    denum = (2*np.pi)**0.5*sigma
    return num/denum

def GaussianNB(X,y):
    '''
    输入X:nxm, y:n,
    返回一个MU:Cxm,SIGMA:Cxm
    '''
    n,m = X.shape
    C = y.max()+1
    MU = np.zeros((C,m))
    SIGMA = np.zeros((C,m))
    for i in range(C):
        indices = np.where(y==i)
        MU[i] = np.mean(X[indices],axis = 0)
        SIGMA[i] = np.std(X[indices],axis = 0)
    return MU,SIGMA

def NBPredict(X, MU, SIGMA):
    '''
    用参数预测X的标签
    '''
    C,m = MU.shape
    n,_ = X.shape
    probs = np.zeros((C,n))
    for i in range(C):
        probs_i = gaussian(X, MU[i], SIGMA[i])
        probs[i] = np.prod(probs_i,axis = 1)
    return np.argmax(probs,axis = 0)


X,y = datasets.load_iris(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, random_state = 3)

MU,SIGMA = GaussianNB(X_train,y_train)
y_pred = NBPredict(X_test,MU,SIGMA)
print("Number of mislabeled points out of a test %d points : %d"
% (X_test.shape[0],(y_test != y_pred).sum()))

Number of mislabeled points out of a test 30 points : 1


### ID3决策树
线性超平面划分的算法多用于处理连续特征, 贝叶斯统计既能处理连续特征也能处理离散特征, 而决策树是一种非常适合处理离散特征的算法. 它的思想非常简单, 在每个树节点, 我们对样本x的一个特征$x_i$做判断, 如果满足$x_i=P_1$就下溯到当前结点的第一个子结点继续判断, 如果满足$x_i=P_2$就下溯到当前结点的第二个子结点继续判断,以此类推, 最后我们到达没有子结点的叶结点, 就可以下决策说这个样本x是某一类样本.  
怎样从训练集得到这样一棵决策树呢,我们要先引入信息熵和信息增益的概念.  
对一个数据集合,我们用下面的方法计算信息熵
$$ Ent(D) = -\sum_{k=1}^n p_k log_2 p_k $$
可以看出, 信息熵是一定是正数, 而且数据集中的样本种类越复杂, 混乱度越高则信息熵越大. 而数据集纯度很高时信息熵很小.  
使用信息熵我们可以衡量一个数据集的纯度, 进而我们可以计算, 在数据集D上, 选择特征a划分得到的几个子数据集的信息增益
$$ Gain(D,a) = Ent(D)-\sum_{v=1}^V \frac{|D_v|}{D}Ent(D_v) $$
上式的意义是, 我们用划分前的信息熵减去, 划分后每一个子数据集的信息熵, 因为子数据集的样本总数不同, 我们用数量的占比加权处理子数据集的信息熵. 一般而言, 信息增益越大, 意味着用a划分的纯度提升越大. 我们就以这种判据, 在每次划分前遍历所有划分可能性, 并选择最好的那一种.  
用信息增益划分树就是ID3算法. 算法从训练集出发, 每次都用当前结点拥有的数据集选择一个属性来做划分, 产生多个子结点, 然后再对子结点继续做划分. 知道所有属性都被划分完, 或者当前结点拥有的数据集都是同一类为止, 这时我们产生叶结点. 每个结点都拥有决策信息, 也就是来到当前结点node的样本x应该被分到node->property类. node->property应该是当前结点拥有的数据集中样本数最多的那个类别.  
有时我们会面临从未在训练集中出现的样本, 这时我们就直接停止下溯, 把它归类到当前非叶结点的property即可. 如果要处理连续特征, ID3给出的方法是对连续特征进行二分, 即排序样本, 并遍历所有中位点, 计算以它为中心进行二分后得到的信息增益.
这里我们实现一个用于连续特征的决策树分类器, 基于ID3算法进行. 因为样本量和维度都较小, 我们每次划分都遍历nxm种划分可能性来寻找最大信息增益. 为了避免过拟合, 我们限制树的高度, 最多划分5次. 

In [2]:
def Ent(labels):
    ent = 0
    Cset = set(labels)
    tot = len(labels)
    for i in Cset:
        p = len(np.where(labels==i)[0])/tot
        ent -= p*np.log2(p)
    return ent

def Gain(X,y):
    '''
    return
    Mid_list: array, shape = (n-1, m)
    表示所有划分用的中值点,Mid_list[i][j]是第j维的第i个划分点
    Gain_list: array, shape = (n-1, m)
    表示以Mid_list[i][j]划分后的信息增益
    '''
    n,m = X.shape
    Mid_list = np.zeros((n-1,m))
    Gain_list = np.zeros((n-1,m))
    Ent_y = Ent(y)
    
    for j in range(m):
        X_sorted = X[np.argsort(X[:,j]),j]
        Mid_list[:,j] = 0.5*X_sorted[:-1]+0.5*X_sorted[1:]
        for i in range(n-1):
            yl = y[X[:,j]<=Mid_list[i,j]]
            yg = y[X[:,j]>Mid_list[i,j]]
            n1,n2,n3 = len(yl),len(yg),len(yl)+len(yg)
            Gain_list[i][j] = Ent_y-n2/n3*Ent(yg)-n1/n3*Ent(yl)
    return Mid_list,Gain_list

In [3]:
from collections import Counter

class Node:
    def __init__(self):
        self.species = 0
        self.dim = 0
        self.mid = 0
        self.leaf = True
        self.left = None
        self.right = None
        
    def grow(self, X, y, max_depth = 4, max_data = 3):
        n,m = X.shape
        cnt = Counter(y)
        self.species = max(set(y),key = lambda x:cnt[x])
        if len(cnt)==1:
            return
        if max_depth==1:
            return
        
        
        Mid_list,Gain_list = Gain(X,y)
        besti_of_each_dim = []
        best_of_each_dim = []
        for j in range(m):
            besti_of_each_dim.append(np.argmax(Gain_list[:,j]))
            best_of_each_dim.append(np.max(Gain_list[:,j]))
        j = np.argmax(np.array(best_of_each_dim))
        i = besti_of_each_dim[j]
        self.mid = Mid_list[i][j]
        self.dim = j
        
        
        indicesl = np.where(X[:,j]<=self.mid)
        indicesg = np.where(X[:,j]>self.mid)
        if len(indicesl[0])<=max_data or len(indicesg[0])<=max_data:
            return
        
        self.left = Node()
        self.right = Node()
        self.left.grow(X[indicesl],y[indicesl],max_depth-1)
        self.right.grow(X[indicesg],y[indicesg],max_depth-1)
        self.leaf = False
        
    def predict(self, x):
        if self.leaf:
            return self.species
        elif x[self.dim]<=self.mid:
            return self.left.predict(x)
        else:
            return self.right.predict(x)

In [4]:
tree = Node()
tree.grow(X_train,y_train)
y_pred = np.array([tree.predict(x) for x in X_test])
print("Number of mislabeled points out of a test %d points : %d"
% (X_test.shape[0],(y_test != y_pred).sum()))

Number of mislabeled points out of a test 30 points : 1
